In [ ]:
#!pip install spotpy

In [ ]:
# imports
from nextgen_calibration_utils import process_usgs_streamflow, run_spotpy, plot_spotpy_results

In [ ]:
# Define inputs for calibration

gage_id = "10109001"
feature_id = 2861391
start_date = "2021-10-01"
end_date = "2022-09-30"
training_start_date = "2021-11-01"

observed_flow_path = f"/shared/examples/calibration/{gage_id}_observed_flow.pkl"
troute_output_path = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}/outputs/troute/troute_output_202110010000.nc"
cfe_dir = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}/config/cat_config/CFE"
data_dir = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}"

# Optional: Retrieve and save observed flow
# process_usgs_streamflow(gage_id, start_date, end_date, output_path=observed_flow_path)

results, best_params = run_spotpy(gage_id, start_date, end_date, training_start_date,
                                  observed_flow_path, troute_output_path, cfe_dir,
                                  data_dir, feature_id, repetitions=5)
plot_spotpy_results(results, training_start_date)

In [ ]:
# Requires MPI setup (Under development)
'''
from mpi4py import MPI
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4)

# Create a parallel view
view = c[:]

params = {'run_spotpy': run_spotpy, 
          'gage_id':gage_id, 
          'start_date':start_date, 
          'end_date':end_date, 
          'training_start_date':training_start_date,
          'observed_flow_path':observed_flow_path, 
          'troute_output_path':troute_output_path, 
          'cfe_dir':cfe_dir,
          'data_dir':data_dir, 
          'feature_id':feature_id}

view.push(params, block=True)

# Run MPI code - use execute() instead of run() for code strings
view.execute('from mpi4py import MPI')

@view.remote(block=True)
def run_spotpy_mpi():

    # Initialize MPI
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    if rank == 0:
        print(f"Running with {comm.Get_size()} MPI processes")
        pass

    # run simulation
    run_spotpy(gage_id, start_date, end_date, training_start_date,
               observed_flow_path, troute_output_path, cfe_dir,
               data_dir, feature_id, repetitions=25)

    if rank == 0:
        print("Optimization complete")
    pass

results = run_spotpy_mpi()
'''